In [1]:
import tweepy
import pandas as pd
import re
import os
import time
import psycopg2
from psycopg2 import sql

In [2]:
API_KEY = "KJfBL1eL2ker6nhicKNhZRYch"
API_SECRET = "AMHdJbAgDgAY3I0yzb505nxiZAy9NSEr8bJnYfG7YGQw5wiOqs"
ACCESS_TOKEN = "1794412103232897024-F0Srvwq3HBVfQIVuGI45UJdCVY4vWN"
ACCESS_SECRET = "V1FlrHRM6A3Foo0pwnovT065M1nGcQSwg5Q0p0vycJeSZ"
BEARER_TOKEN = "AAAAAAAAAAAAAAAAAAAAAFzHygEAAAAA1hloN%2Bf4XNj2b6Dh0wUr5K2wjww%3Dllj5Ik3tp6USoSVHcGnU5hRdDJt8xmywFgpTig0BloS1z1ZqpT"

In [3]:
client = tweepy.Client(bearer_token=BEARER_TOKEN)

In [4]:

# PostgreSQL database credentials
DB_NAME = "postgres"
DB_USER = "postgres"
DB_PASSWORD = "root"
DB_HOST = "localhost"
DB_PORT = "5432"

In [5]:
# Connect to PostgreSQL database
conn = psycopg2.connect(
    dbname=DB_NAME,
    user=DB_USER,
    password=DB_PASSWORD,
    host=DB_HOST,
    port=DB_PORT
)
cursor = conn.cursor()

In [6]:
# Create table if not exists
cursor.execute("""
    CREATE TABLE IF NOT EXISTS twitter_profiles (
        id SERIAL PRIMARY KEY,
        username TEXT UNIQUE,
        bio TEXT,
        followers_count INTEGER,
        following_count INTEGER,
        location TEXT,
        website TEXT
    )
""")
conn.commit()


In [10]:
# Load CSV file containing Twitter profile links
csv_file_path = r"D:\\projects\\ML\\webscrap\\twitter_links1.csv"
df = pd.read_csv(csv_file_path)

In [13]:
print(df.columns)

Index(['Links '], dtype='object')


In [14]:
def extract_username(url):
    match = re.search(r"twitter\.com/([^/?]+)", str(url))
    return match.group(1) if match else None

df["Username"] = df["Links "].apply(extract_username)

In [15]:
def fetch_user_data(username):
    while True:
        try:
            user = client.get_user(username=username, user_fields=["description", "public_metrics", "location", "url"])
            return {
                "Username": username,
                "Bio": user.data.description if user.data.description else "",
                "Followers Count": user.data.public_metrics["followers_count"],
                "Following Count": user.data.public_metrics["following_count"],
                "Location": user.data.location if user.data.location else "",
                "Website": user.data.url if user.data.url else ""
            }
        except tweepy.TooManyRequests as e:
            reset_time = int(e.response.headers.get("x-rate-limit-reset", time.time() + 60))
            wait_time = reset_time - time.time()
            print(f"Rate limit exceeded. Waiting {wait_time:.0f} seconds...")
            time.sleep(wait_time + 1)
        except Exception as e:
            print(f"Error fetching data for {username}: {e}")
            return None


In [16]:
for username in df["Username"].dropna().head(3):
    user_data = fetch_user_data(username)
    if user_data:
        cursor.execute("""
            INSERT INTO twitter_profiles (username, bio, followers_count, following_count, location, website)
            VALUES (%s, %s, %s, %s, %s, %s)
            ON CONFLICT (username) DO UPDATE
            SET bio = EXCLUDED.bio, 
                followers_count = EXCLUDED.followers_count, 
                following_count = EXCLUDED.following_count, 
                location = EXCLUDED.location, 
                website = EXCLUDED.website
        """, (user_data["Username"], user_data["Bio"], user_data["Followers Count"], user_data["Following Count"], user_data["Location"], user_data["Website"]))
        conn.commit()
        print(f"Fetched and stored data for {username}")
        time.sleep(15)


Fetched and stored data for GTNUK1
Fetched and stored data for whatsapp
Fetched and stored data for aacb_CBPTrade


In [17]:
# Close connection
cursor.close()
conn.close()
print("Data successfully stored in PostgreSQL database.")

Data successfully stored in PostgreSQL database.
